# Cells for Multiple Customer Classes

## Basis in the Queueing Network Analyzer

### Input information

For the system, the required input parameters are:
- $n = $ total number of nodes
- $r = $ number of classes (number of routes)
and include variables:
- $m_j = $ number of servers at node j

For the $k$th class of customer, the additional inputs are:
- $n_k = $ the total number of nodes visited by class $k$
- $n_{kj} = $ the $j$th node for class $k$

and includes variables:
- $\hat{\lambda}_k = $ class $k$ production rate (through the entire system)

## Derivation of GP-Compatible Model

Conserve all of the different arrival streams of customers:
$$ \bar{\lambda} \geq \sum_{i=1}^n \lambda_i $$
This is valid due to the upward pressure on the individual process rates

Find fractional behavior of each of the customer classes
$$ x_i = \frac{\lambda_i}{\bar{\lambda}}, \quad \forall i \in [1,n] $$

For each customer class, Little's law applies
$$ L_i = \lambda_i W_i $$
where the flow time $W_i$ is modeled as:
$$W_i \geq \bar{W_q} + t_{s,i}$$

__NOTE__

The GP formulation of the single-class QNA cell does not solve for L

_ISSUE_
This constraint is not tight. 

### Cell Aggregation
Cell service and variation are approximated as aggregations of the different classes

### Aggregate Cell performance
Based on QNA Model

The aggregate queueing time is modeled with Kingmann's equation
$$ \bar{W_q} \geq \left(\frac{\rho}{\alpha}\right) \frac{\bar{c_s^2}+\bar{c_a^2}}{2} \frac{\bar{\tau_s}}{mk} $$
with the usual relaxation for $\alpha$
$$ 1 \geq \alpha + \rho $$

The departure coefficient of variation is approximated by:
$$ c_d^2 \geq \alpha\bar{c_a^2} + \rho\bar{c_s^2} $$

The aggregate prodcution rate of the cell, the aggregate utilization, the aggregate processing time and the server count $m$ are also related by:
$$ \bar{\rho} = \frac{\bar{\lambda}\bar{\tau_s}}{m} $$
which can be rewritten as the constraint
$$ \bar{\lambda} \leq \frac{m\bar{\rho}}{\bar{\tau_s}} $$

The aggregate process time is the weighted average of the class service times:
$$ \bar{t_s} \geq \sum{x_it_{s,i}} $$

This constraint may also be written:
$$ \bar{t_s} \geq \frac{1}{\bar{\lambda}} \sum{x_it_{s,i}} $$
and here we see the problem that if $\bar{\lambda}$ is not upper-bounded, this will not be tight. We need to put downward pressure on $\bar{\lambda}$. Alternatively, if we can assume that the class arrival rates $\lambda_i$ are fixed variables, we can calculate $\bar{\lambda}$ and this is not an issue. 

__Approximate Service Variation as the Geometric Mean__

try a new approximation for the aggregate coefficient of variation:
$$ \bar{t_s^2}\bar{c_s^2} = \sqrt{\left(x_1c_{s,1}^2t_{s,1}^2\right)\left(x_2c_{s,2}^2t_{s,2}^2\right) }$$
The geometric mean is always above the artithmatic mean

If, however, we can assume that $\bar{\lambda}$ is a fixed variable, as noted above, we can use the relaxed constraint version as 
$$ \bar{t_s^2}\bar{c_s^2} \geq \sum x_i t_s^2 c_{s,i}^2 $$

_Compare to Mixture of Squared Coefficients of Variation_
$$\bar{c_s^2} \approx \sum{x_i c_{s,i}^2} \forall i$$

### Final GP Model

#### Calculated Fixed Variables

__Aggregate Production Rate__
$$ \bar{\lambda} = \sum \lambda_i  \forall i $$

__Mixture Parameters__
$$ x_i = \frac{\lambda_i}{\sum_{j \in m} \lambda_j} \quad \forall i $$

#### Aggregate Service Performance Variables

__Aggregate Service Time__
$$ \bar{t_s} \geq \sum t_i \quad \forall i $$

__Aggregate Arrival Behavior__
$$ \bar{c_a^2} \geq \sum x_i c_{a,i}^2 $$

__Inventory at Service__ (maybe)
$$ \bar{L} \geq \sum L_i \forall i$$

#### Aggregate QNA Cell Performance

__Queueing Time__
$$ \bar{W_q} \geq \frac{\bar{\rho}}{\bar{\alpha}}
\frac{\bar{c_a^2 + c_s^2}}{2} \frac{\bar{\tau_s}}{m} $$

__Utilization Balance__
$$ 1 \geq \bar{\rho} + \bar{\alpha} $$

__Departure Variation__

#### Add Class Cell constraints

__Queueing Inventory__
$$ L_{q,i} = \lambda_i \bar{W_q} $$

__Departure Variation__
$$ c_{d,i}^2 \geq \bar{c_d^2} \quad \forall i $$

__Queueing Time__
$$ W_{q,i} \geq \bar{W_q} $$

__Cell Flow Time__
$$ W_i \geq W_{q,i} + \tau_i $$

__Cell Service Variation__
$$c_{s,i}^2 \geq \left(c_{s,process,i}\right)^2 $$

#### Class constraints to keep

__Total Inventory__
$$ L_i \geq \lambda_i \left(\bar{W_q} + \tau_{s,i} \right) $$

## GPX Implementation

In [25]:
from gpkit import Variable, Model, VectorVariable, units

In [2]:
import numpy as np

In [4]:
from gpx.primitives import Process

In [6]:
# from gpkit import unitstest.get_constraint(test.W)

In [7]:
ts1 = Variable('t_{s,1}', 30, 'min', 'service time')
ts2 = Variable('t_{s,2}', 45, 'min', 'service time')

In [8]:
processes = [Process(time=ts1), Process(time=ts2)]

### Prototype Implementation

In [ ]:
# Upper Unbounded
#     ---------------
#     m, W_bar, 
    
#     Lower Unbounded
#     ---------------
#     c2a_bar

In [104]:
class SlimMultiCell(Model):
    '''a slim model for a multi-class cell
    
    Variables
    ---------
    m    [count]    workstation count
    c2d  [-]        Departure coefficient of variation
    
    Upper UB
    ---------------
    m, c2d, L_bar
    
    Lower UB
    ---------------
    c2a
    
    
    '''
    def setup(self, processes=[]):
        self.processes = processes
        # declare variables
        ## variables
        m = self.m = Variable('m', 'count', 'workstation count')
        c2d = self.c2d = Variable('c_d^2', '-', 'departure squared coefficient of variation')
        
        ## aggregate variables
        Wq_bar = self.Wq_bar = Variable('\\bar{W_q}', 'min', 'queueing time')
        L_bar = self.L_bar = Variable('\\bar{L}', 'count', 'total inventory')
        ts_bar = self.ts_bar = Variable('\\bar{t_s}', 'min', 'service time')
        lam_bar = self.lam_bar = Variable('\\bar{\\lambda}', 'count/hr', 'total production rate')
        rho_bar = self.rho_bar = Variable('\\bar{\\rho}', '-', 'utilization')
        alpha_bar = self.alpha_bar = Variable('\\bar{\\alpha}', '-', '1-rho')
        c2a_bar = self.c2a_bar = Variable('\\bar{c^2_a}', '-', 'Arrival coefficient of variation squared')
        c2s_bar = self.c2s_bar = Variable('\\bar{c^2_s}', '-', 'Service coefficient of variation squared')
        W_bar = self.W_bar = Variable('\\bar{W}', 'min', 'aggregate part flow time')
        
        ## vector variables
        n = len(processes)
        
        x = self.x = VectorVariable(n, 'x', '-', 'fraction of production rate')
        lam = self.lam = VectorVariable(n, '\\lambda', 'count/hr', 'production rate')
        ts = self.ts = np.array([p.t for p in processes])
        c2s = self.c2s = np.array([p.cv**2 for p in processes])
        c2a = self.c2a = VectorVariable(n, 'c^2_a', '-', 'arrival coefficient of variation')
        
        constr = []
        
        # aggregation calculations
        constr.extend([
            ts_bar >= np.sum(x*ts),
            ts_bar**2*c2s_bar >= np.sum(x*c2s*ts**2),
            c2a_bar >= np.sum(x*c2a),
        ])
        
        # QNA Constraints
        constr.extend([
            W_bar >= Wq_bar + ts_bar,
            L_bar >= lam_bar*W_bar,
            
            Wq_bar >= (rho_bar/alpha_bar)*(c2a_bar + c2s_bar)/2*ts_bar/m,
            1 >= alpha_bar + rho_bar,
            c2d >= alpha_bar*c2a_bar + rho_bar*c2s_bar,
            lam_bar <= rho_bar*m/ts_bar,   # check that this is not redundant
        ])
        
        # classes constraints
        constr.extend([
            x*lam_bar >= lam,
            1 >= np.sum(x),
        ])
        
        return [constr, processes]

#### TEST: Slim Implementation

In [105]:
m = SlimMultiCell(processes)

In [106]:
m.substitutions.update({
    processes[0].t : 30*units('min'),
    processes[1].t : 60*units('min'),
    processes[0].cv : 1,
    processes[1].cv : 1,
    m.lam : [2, 1],   # should correspond to m ~ 2
    m.c2a : [1, 1],
})

In [107]:
# Enforce things that would usually have downward pressure from elsewhere
m.cost = m.m + m.L_bar + m.c2d*units('count')

In [108]:
print(m.solve().table())

Using solver 'cvxopt'
 for 15 free variables
  in 17 posynomial inequalities.
Solving took 0.0222 seconds.

Optimal Cost
------------
 7.364

Free Variables
--------------
              | Process
       \sigma : 30                      [min]      standard deviation of process time

              | Process1
       \sigma : 60                      [min]      standard deviation of process time

              | SlimMultiCell19
      \bar{L} : 2.94                    [count]    total inventory
    \bar{W_q} : 18.81                   [min]      queueing time
      \bar{W} : 58.81                   [min]      aggregate part flow time
 \bar{\alpha} : 0.4029                             1-rho
\bar{\lambda} : 3                       [count/hr] total production rate
   \bar{\rho} : 0.5971                             utilization
  \bar{c^2_a} : 1                                  Arrival coefficient of variation squared
  \bar{c^2_s} : 1.125                              Service coefficient of variat

Works with fixed arrival CV
- adjust to use feedback from the cell

In [115]:
try:
    del m.substitutions[m.c2a]
except KeyError:
    print('skipping')
    
m.substitutions[m.processes[0].cv] = 0.5
m.substitutions[m.lam[1]] = 2

skipping


In [117]:
# loop the variation back around
m.append(m.c2a >= m.c2d)

In [113]:
m.cost = m.m + m.L_bar

In [116]:
print(m.solve().table())

Using solver 'cvxopt'
 for 17 free variables
  in 19 posynomial inequalities.
Solving took 0.0226 seconds.

Optimal Cost
------------
 8.755

Free Variables
--------------
              | Process
       \sigma : 15                      [min]      standard deviation of process time

              | Process1
       \sigma : 60                      [min]      standard deviation of process time

              | SlimMultiCell19
      \bar{L} : 4.175                   [count]    total inventory
    \bar{W_q} : 17.62                   [min]      queueing time
      \bar{W} : 62.62                   [min]      aggregate part flow time
 \bar{\alpha} : 0.345                              1-rho
\bar{\lambda} : 4                       [count/hr] total production rate
   \bar{\rho} : 0.655                              utilization
  \bar{c^2_a} : 0.9444                             Arrival coefficient of variation squared
  \bar{c^2_s} : 0.9444                             Service coefficient of variat

### Additional Constraints to Return

__By Class__

- Flow Time
- Inventory
- Utilization

### Full Implementation

In [ ]:
class CustomerClass(Model):
    '''A class of customer
    
    Attributes
    ----------
    process_chain : 
        the ordered list of processes to define 
    '''

In [287]:
class MultiCell(Model):
    '''a slim model for a multi-class cell
    
    Variables
    ---------
    m    [count]    workstation count
    c2d  [-]        Departure coefficient of variation
    
    Upper UB
    ---------------
    m, c2d, L_bar
    
    Lower UB
    ---------------
    c2a
    
    
    '''
    def setup(self, processes=[], class_performance=True, aggregate_performance=False):
        '''model setup
        
        Arguments
        ---------
        processes : list of gpx.primitives.Process
            the input process
        
        '''
        self.processes = processes
        # declare variables
        ## variables
        m = self.m = Variable('m', 'count', 'workstation count')
        c2d = self.c2d = Variable('c_d^2', '-', 'departure squared coefficient of variation')
        
        ## aggregate variables
        Wq_bar = self.Wq_bar = Variable('\\bar{W_q}', 'min', 'queueing time')
        L_bar = self.L_bar = Variable('\\bar{L}', 'count', 'total inventory')
        ts_bar = self.ts_bar = Variable('\\bar{t_s}', 'min', 'service time')
        lam_bar = self.lam_bar = Variable('\\bar{\\lambda}', 'count/hr', 'total production rate')
        rho_bar = self.rho_bar = Variable('\\bar{\\rho}', '-', 'utilization')
        alpha_bar = self.alpha_bar = Variable('\\bar{\\alpha}', '-', '1-rho')
        c2a_bar = self.c2a_bar = Variable('\\bar{c^2_a}', '-', 'Arrival coefficient of variation squared')
        c2s_bar = self.c2s_bar = Variable('\\bar{c^2_s}', '-', 'Service coefficient of variation squared')
        W_bar = self.W_bar = Variable('\\bar{W}', 'min', 'aggregate part flow time')
        
        ## vector variables
        n = len(processes)
        
        x = self.x = VectorVariable(n, 'x', '-', 'fraction of production rate')
        lam = self.lam = VectorVariable(n, '\\lambda', 'count/hr', 'production rate')
        ts = self.ts = np.array([p.t for p in processes])
        c2s = self.c2s = np.array([p.cv**2 for p in processes])
        c2a = self.c2a = VectorVariable(n, 'c^2_a', '-', 'arrival coefficient of variation')
        W = self.W = VectorVariable(n, 'W', 'min', 'part flow time')
        L = self.L = VectorVariable(n, 'L', 'count', 'part inventory count')
        
        constr = []
        
        # aggregation calculations
        constr.extend([
            ts_bar >= np.sum(x*ts),
            ts_bar**2*c2s_bar >= np.sum(x*c2s*ts**2),
            c2a_bar >= np.sum(x*c2a),
        ])
        
        # QNA Constraints
        constr.extend([
#             W_bar >= Wq_bar + ts_bar,
#             L_bar >= lam_bar*W_bar,
            
            Wq_bar >= (rho_bar/alpha_bar)*(c2a_bar + c2s_bar)/2*ts_bar/m,
            1 >= alpha_bar + rho_bar,
            c2d >= alpha_bar*c2a_bar + rho_bar*c2s_bar,
            lam_bar <= rho_bar*m/ts_bar,   # check that this is not redundant
        ])
        
        # classes constraints
        constr.extend([
            x*lam_bar == lam,
            1 >= np.sum(x),
        ])
        
        if aggregate_performance:
            # extend with aggregate performance constraints (little's law)
            constr.extend([
                W_bar >= Wq_bar + ts_bar,
                L_bar >= lam_bar*W_bar,
            ])
            
        if class_performance:
            # extend with little's law performance constraints by customer class
            constr.extend([
                L >= x*lam_bar*(Wq_bar+ts),
            ])
        
        return [*constr, *processes]

#### TEST

In [288]:
m2 = MultiCell(processes=processes, class_performance=True, aggregate_performance=False)
m2.cost = m2.m + np.sum(m2.L)

In [245]:
m2 = MultiCell(processes=processes, class_performance=True, aggregate_performance=True)
m2.cost = m2.m + m2.L_bar + np.sum(m2.L)

In [282]:
m2 = MultiCell(processes=processes, class_performance=False, aggregate_performance=True)
m2.cost = m2.m + m2.L_bar

In [283]:
# m2.extend(m2.get_constraint(m2.L_bar))

In [289]:
m2.substitutions.update({
    processes[0].t : 30*units('min'),
    processes[1].t : 60*units('min'),
    processes[0].cv : 1,
    processes[1].cv : 1,
    m2.lam : [2, 1],   # should correspond to m ~ 2
})

In [290]:
m2.append(m2.c2a >= m2.c2d)

In [286]:
print(m2.solve().table())

Using solver 'cvxopt'
 for 17 free variables
  in 21 posynomial inequalities.
Solving took 0.0283 seconds.

Optimal Cost
------------
 6.345

Free Variables
--------------
              | MultiCell33
      \bar{L} : 2.974                   [count]    total inventory
    \bar{W_q} : 19.49                   [min]      queueing time
      \bar{W} : 59.48                   [min]      aggregate part flow time
 \bar{\alpha} : 0.4066                             1-rho
\bar{\lambda} : 3                       [count/hr] total production rate
   \bar{\rho} : 0.5934                             utilization
  \bar{c^2_a} : 1.125                              Arrival coefficient of variation squared
  \bar{c^2_s} : 1.125                              Service coefficient of variation squared
    \bar{t_s} : 40                      [min]      service time
        c_d^2 : 1.125                              departure squared coefficient of variation
            m : 3.37                    [count]    workst

when we go to the class performance, $\bar{\lambda}$ gets very large and $x_i$ becomes small as does $\bar{\tau_s}$

#### Test as SP

In [291]:
from gpkit import SignomialsEnabled

In [292]:
with SignomialsEnabled():
    m2.append(np.sum(m2.x) >= 1)

In [294]:
print(m2.localsolve().table())

Starting a sequence of GP solves
 for 3 free variables
  in 1 locally-GP constraints
  and for 18 free variables
       in 23 posynomial inequalities.
Solving took 0.105 seconds and 3 GP solves.

Optimal Cost
------------
 6.345

Free Variables
--------------
              | MultiCell34
    \bar{W_q} : 19.49                   [min]      queueing time
 \bar{\alpha} : 0.4066                             1-rho
\bar{\lambda} : 3                       [count/hr] total production rate
   \bar{\rho} : 0.5934                             utilization
  \bar{c^2_a} : 1.125                              Arrival coefficient of variation squared
  \bar{c^2_s} : 1.125                              Service coefficient of variation squared
    \bar{t_s} : 40                      [min]      service time
        c_d^2 : 1.125                              departure squared coefficient of variation
            m : 3.37                    [count]    workstation count
            L : [ 1.65      1.32     ]  [co

can solve as SP if $\mathbf{x}$ is a free variable and $\lambda_i$ are fixed variables.

_What if $x_i$ are fixed variables?_

#### Test as GP with fixed variable $x_i$

In [305]:
m3 = MultiCell(processes=processes)
m3.cost = m3.m*10 + np.sum(m3.L)

In [306]:
m3.append(m3.c2a == m3.c2d)

In [307]:
m3.substitutions.update({
    processes[0].t : 30*units('min'),
    processes[1].t : 60*units('min'),
    processes[0].cv : 1,
    processes[1].cv : 1,
    m3.x : [2.0/3.0, 1.0/3.0],   # should correspond to m ~ 2
    m3.lam_bar : 3,   # if only have x, also have to constrain lambda
})

In [308]:
print(m3.solve().table())

Using solver 'cvxopt'
 for 16 free variables
  in 22 posynomial inequalities.
Solving took 0.0256 seconds.

Optimal Cost
------------
 30.58

Free Variables
--------------
             | MultiCell36
   \bar{W_q} : 78.36                   [min]      queueing time
\bar{\alpha} : 0.1889                             1-rho
  \bar{\rho} : 0.8111                             utilization
 \bar{c^2_a} : 1.125                              Arrival coefficient of variation squared
 \bar{c^2_s} : 1.125                              Service coefficient of variation squared
   \bar{t_s} : 40                      [min]      service time
       c_d^2 : 1.125                              departure squared coefficient of variation
           m : 2.466                   [count]    workstation count
           L : [ 3.61      2.31     ]  [count]    part inventory count
     \lambda : [ 2         1        ]  [count/hr] production rate
       c^2_a : [ 1.13      1.13     ]             arrival coefficient of var

if $x_i$ is the fixed variable, can solve as GP

use evaluated fixed variables to get $x_i$

### Evaluated Fixed Variables

#### Test from GPkit Documentation

In [324]:
x = Variable("x", "hours")
t_day = Variable("t_{day}", 12, "hours")
t_night = Variable("t_{night}", lambda c: 24 - c[t_day], "hours")

the `lambda` function evaluates the variable from the substitution list

In [325]:
m = Model(x, [x >= t_day, 24*units('hours')>= t_day + t_night, x >= t_night])

In [326]:
print(m.solve().table())

Using solver 'cvxopt'
 for 1 free variables
  in 3 posynomial inequalities.
Solving took 0.00826 seconds.

Optimal Cost
------------
 12

Free Variables
--------------
x : 12  [hr]

Fixed Variables
---------------
  t_{day} : 12  [hr]
t_{night} : 12  [hr]

Most Sensitive Constraints
--------------------------
  +0.5 : x >= t_{day}
  +0.5 : x >= t_{night}



#### Calculate $x_i$ and $\bar{\lambda}$

In [310]:
lam_sys = Variable('\\lambda_{system}', 1.0, 'count/hr', 'system production rate')
qty = VectorVariable(2, 'n_{part}', [2,1], 'count', 'part quanities')
lam_bar = Variable('\\lambda', lambda c: np.sum(c[qty])*c[lam_sys], 'count/hr', 'min flow')
x = VectorVariable(2, 'x', lambda)

In [312]:
test = Model(x, [x >= lam_bar,
                ])

#### Calculate Based in Models

In [5]:
from gpkit import Model, Variable, VectorVariable

In [9]:
import numpy as np

In [7]:
lam = VectorVariable( 2, '\\lambda', [3,1], 'count/hr', 'class production rate')

In [11]:
lambar = Variable('\\bar{\\lambda}', lambda c : np.sum(c[lam]), 'count/hr', 'overall production rate')

In [12]:
# simple model solves to min lambar
m = Model()

In [13]:
m.cost = (lambar + np.sum(lam))

In [15]:
m.solve()

ValueError: Geometric Program is not fully bounded:
  \bar{\lambda} has no lower bound
  \lambda[0] has no lower bound
  \lambda[1] has no lower bound

In [ ]:
# create a simple optimization to min the sum of x

In [327]:
class MultiCellEvalFixed(Model):
    '''A Multi-Class Cell with Evaluted Fixed Variables
    '''
    pass

#### Old implementation

In [112]:
class MultiCell(Model):
    def setup(self, processes=[], return_processes=True, **kwargs):
        '''
        Keyword Arguments
        -----------------
        processes : iterable of processes
        
        Variables
        ---------
        m    [count]    workstation count
        c2d  [-]        Departure coefficient of variation
        
        Aggregate Performance Variables
        -------------------------------
        Wq_bar    [min]          queueing time
        Lq_bar    [count]        customers in queue
        lam_bar   [count/min]    production rate
        rho_bar   [-]            Utilization
        alpha_bar [-]            (1-rho)
        ts_bar    [min]          Service time
        c2a_bar   [-]            Arrival coefficient of variation
        c2s_bar   [-]            Service coefficient of variation
        
        Vector Variables
        ----------------
        lam    [count/min]    Production rate of each of the streams
        x      [-]            Fraction of parts
        W      [min]          Total flow time
        '''
        # Variables
        m = self.m = Variable('m', 'count', 'workstation count')
        
        # Aggregate Variables
        Wq_bar = self.Wq_bar = Variable('\\bar{W_q}', 'min', 'queueing time')
        ts_bar = self.ts_bar = Variable('\\bar{t_s}', 'min', 'service time')
        lam_bar = self.lam_bar = Variable('\\bar{\\lambda}', 'count/hr', 'total production rate')
        rho_bar = self.rho_bar = Variable('\\bar{\\rho}', '-', 'utilization')
        alpha_bar = self.alpha_bar = Variable('\\bar{\\alpha}', '-', '1-rho')
        c2a_bar = self.c2a_bar = Variable('\\bar{c^2_a}', '-', 'Arrival coefficient of variation squared')
        c2s_bar = self.c2s_bar = Variable('\\bar{c^2_s}', '-', 'Service coefficient of variation')
        c2d_bar = self.c2d_bar = Variable('\\bar{c^2_d}', '-', 'Departure coefficient of variation')
        
        constr = []
        
        # add QNA constraints
        constr.extend([
            Wq_bar >= (rho_bar/alpha_bar)*(c2a_bar + c2s_bar)/2*ts_bar/m,
            1 >= alpha_bar + rho_bar,
            c2d_bar >= alpha_bar*c2a_bar + rho_bar*c2s_bar,
            lam_bar == rho_bar*m/ts_bar,
        ])
        
        # inputs based on customer classes
        n = self.n = len(processes)
        lam = self.lam = VectorVariable(n, '\\lambda', 'count/hr', 'unit production rate')
        x = self.x = VectorVariable(n, 'x', '-', 'fraction of dedicated production')
        W = self.W = VectorVariable(n, 'W', 'min', 'total flow time')
        L = self.L = VectorVariable(n, 'L', 'count', 'total inventory count at cell')
        c2a = self.c2a = VectorVariable(n, 'c^2_a', '-', 'squared coefficient of variation of arrival process')
        ts = self.ts = np.array([p.t for p in processes])
        c2s = self.c2s = np.array([p.cv**2 for p in processes])
        
        constr.extend([
            lam_bar >= np.sum(lam),
            x == lam/lam_bar,
#             lam <= x*lam_bar,
            1 >= np.sum(x)
        ])
        
        # constraints to aggregate
        constr.extend([
            ts_bar >= np.sum(x*ts),
#             c2s_bar*ts_bar**2 >= np.sum(x*c2s*ts**2),
            c2s_bar*ts_bar**2 == np.multiply(x,np.multiply(ts**2, c2s))**(1/n),
            c2a_bar >= np.sum(x*c2a),
        ])
        
        #TODO: need a constraint on x adding up to 1
        
        if return_processes:
            constr.extend([*processes])
        
        return constr
    
    def get_constraint(self, var, bound='lower'):
        '''get a particular constraint that is not internally bounded by the QNA cell
        Should be used when formulating constraints elsewhere that will provide opposing pressure
        '''
        if var == self.W:
            # flow time
            return (self.W >= self.Wq_bar + self.ts)
        
        if var == self.L:
            # inventory
            pass
        if var == self.Lq:
            # inventory in queue
            pass
        

In [113]:
test = MultiCell(processes)

test.cost = test.m

test.substitutions[test.lam] = [2, 3]
test.substitutions[test.c2a] = [0.25, 0.25]
test.substitutions[processes[0].cv] = 0.5
test.substitutions[processes[1].cv] = 0.5

test.append(test.Wq_bar <= 120*units('min'))

Infeasible monomial equality: Cannot convert from 'MultiCell22.\bar{c^2_s}*MultiCell22.\bar{t_s}² [min²]' to '(MultiCell22.x[0]*t_{s,1}²*Process10.cv²)^0.5 [min]'
Infeasible monomial equality: Cannot convert from 'MultiCell22.\bar{c^2_s}*MultiCell22.\bar{t_s}² [min²]' to '(MultiCell22.x[0]*t_{s,1}²*Process10.cv²)^0.5 [min]'
Infeasible monomial equality: Cannot convert from 'MultiCell22.\bar{c^2_s}*MultiCell22.\bar{t_s}² [min²]' to '(MultiCell22.x[1]*t_{s,2}²*Process11.cv²)^0.5 [min]'


ValueError: An ArrayConstraint was created with elements of numpy.bool_
Full constraint: ([MultiCell22.x[0]*t_{s,1}²*Process10.cv², MultiCell22.x[1]*t_{s,2}²*Process11.cv²])^0.5 = MultiCell22.\bar{c^2_s}*MultiCell22.\bar{t_s}²

## Implementation with scalability

- a MP-QNA cell needs to know the different products

_Paradigm_
- define the `system` which has many `products`
- the products all have their own production flows which include cells, processes, tools, etc.
- the system figures out the overlapping cells
- use the line information
- the product.line has the routing information
  - rate information

Access the arrival information: `system.cell`
- combines all of the product cells
- in the MP context, the queueing time for the product cells is the same (since we do not assume any sort of priority queueing at this phase)

In [1]:
from gpx.manufacturing import QNACell, FabLine

In [3]:
from gpx import Model

In [4]:
from gpkit import Variable

In [5]:
import numpy as np

### System

a system in the MC context has all of the different classes (products) in the same collection of cells


[ ] the system will have to manage all of the shared cells:
- combining arrivals

In [ ]:
class MCSystem(Model):
    '''a system of one or more classes is in a multi-class context
    
    Attributes
    ----------
    
    Variables
    ---------
    '''
    
    def setup(self, classes=[]):
        '''
        
        Arguments
        ---------
        
        '''
        if len(classes)

### Class (Product)

basically aything that describes the class
- production lines
- secondary lines
- feeder lines
- rate

In [2]:
class MClass(Model):
    '''a single product in a multi-product context
    
    Attributes
    ----------
    line : gpx.MultiClass.MCLine
        
    cells : 
    
    Variables
    ---------
    lambda    [count/hr]    class production rate
    '''
    def setup(self, line):
        '''
        Arguments
        ---------
        line :
        '''
        pass
    

### Cell

the `MCCell` is only used in the system context

has to figure out the balance of the different classes:
- calculating $x$ for all of the classes
  - we'll have to assume that the rate for the classes are inputs
  - in the solve context where we seek to maximize rate, we'll have to know $x$ for the splits among the products ahead of time

#### Constraints at `MCCell` vs `QNACell`

__Constraints on `QNACell` to be added at `MCCell`__
$$c_{d,i}^2 \geq \bar{c_d^2} $$
$$W_{q,i} \geq \bar{W_q} $$

__Valid `QNACell` constraints__
- parallel capacity $k$
- 


__Variables from `QNACell` that we'd just like to replace with those from `MCCell`__
- $c^2_d$
- $W_q$
- 

__Variables from `QNACell` that become superfluous__
- $m$
- $\rho$ (though we might be able to see the partial utilization of the shared cell)
- $\alpha$


#### Implementation

In [ ]:
class MCell(Model):
    def setup(self, classcells=[] return_processes=True, **kwargs):
        '''
        Keyword Arguments
        -----------------
        classcells : OrderedDict of {class : cell}
            can ask the
            QUESTION: does this need to be an ordered dict?
        
        Variables
        ---------
        m    [count]    workstation count
        c2d  [-]        Departure coefficient of variation
        
        Aggregate Performance Variables
        -------------------------------
        Wq_bar    [min]          queueing time
        Lq_bar    [count]        customers in queue
        rho_bar   [-]            Utilization
        alpha_bar [-]            (1-rho)
        ts_bar    [min]          Service time
        c2a_bar   [-]            Arrival coefficient of variation
        c2s_bar   [-]            Service coefficient of variation
        
        Calculated Variables
        --------------------
        lam_bar   [count/min]    production rate
        
        
        Variables Assigned to Single-Class Cells
        ----------------------------------------
        x    [-]      Fraction of parts in the multi-class cell
        W    [min]    Total flow time
        
        
        Vector (by class) Variables
        ---------------------------        
        W      [min]          Total flow time
        c2a    [-]            Arrival squared coefficient of variation
        c2s    [-]            Service squared coefficient of variation
        L      [count]        Average customers in the queue and in service
        '''
        # Variables
        m = self.m = Variable('m', 'count', 'workstation count')
        
        # Aggregate Variables
        Wq_bar = self.Wq_bar = Variable('\\bar{W_q}', 'min', 'queueing time')
        ts_bar = self.ts_bar = Variable('\\bar{t_s}', 'min', 'service time')
        lam_bar = self.lam_bar = Variable('\\bar{\\lambda}', 'count/hr', 'total production rate')
        rho_bar = self.rho_bar = Variable('\\bar{\\rho}', '-', 'utilization')
        alpha_bar = self.alpha_bar = Variable('\\bar{\\alpha}', '-', '1-rho')
        c2a_bar = self.c2a_bar = Variable('\\bar{c^2_a}', '-', 'Arrival coefficient of variation squared')
        c2s_bar = self.c2s_bar = Variable('\\bar{c^2_s}', '-', 'Service coefficient of variation')
        c2d_bar = self.c2d_bar = Variable('\\bar{c^2_d}', '-', 'Departure coefficient of variation')
        
        # Calculated Variables
        # Find the X proportions
        for c in classcells.values():
            c.x = Variable('x',
                           lambda m, C=c: m[c.lam]/np.sum([m[cc.lam] for cc in classcells.values()]),
                           '-',
                           'arrival rate mixture parameter'                          
                          )
        
        # Vectors of class cell variables
        x = [c.x for c in classcells.values()]
        ts = [c.t for c in classcells.values()]
        
        constr = []
        
        # Calculated Aggregate Variables
        constr.append(lam_bar >= np.sum([c.lam for classcells.values()]))   # cell rate exceeds required rate for class cells
        
        # add QNA constraints for the Aggregate Cell
        constr.extend([
            Wq_bar >= (rho_bar/alpha_bar)*(c2a_bar + c2s_bar)/2*ts_bar/m,
            1 >= alpha_bar + rho_bar,
            c2d_bar >= alpha_bar*c2a_bar + rho_bar*c2s_bar,
            lam_bar == rho_bar*m/ts_bar,
        ])
        
        # inputs based on customer classes
#         lam = self.lam = VectorVariable(n, '\\lambda', 'count/hr', 'unit production rate')
#         x = self.x = VectorVariable(n, 'x', '-', 'fraction of dedicated production')
#         W = self.W = VectorVariable(n, 'W', 'min', 'total flow time')
#         L = self.L = VectorVariable(n, 'L', 'count', 'total inventory count at cell')
#         c2a = self.c2a = VectorVariable(n, 'c^2_a', '-', 'squared coefficient of variation of arrival process')
#         ts = self.ts = np.array([p.t for p in processes])
#         c2s = self.c2s = np.array([p.cv**2 for p in processes])
        
        # constraints to aggregate
        constr.extend([
            ts_bar >= np.sum(),
#             c2s_bar*ts_bar**2 >= np.sum(x*c2s*ts**2),
            c2s_bar*ts_bar**2 == np.multiply(x,np.multiply(ts**2, c2s))**(1/n),
            c2a_bar >= np.sum(x*c2a),
        ])
        
        if return_processes:
            constr.extend([*processes])
            
        #TODO: constrain all of the cell objects in the classes
        # loop through classes
        # find the corresponding cell
        # constrain class L
        
        return constr
    
    def get_constraint(self, var, bound='lower'):
        '''get a particular constraint that is not internally bounded by the QNA cell
        Should be used when formulating constraints elsewhere that will provide opposing pressure
        '''
        if var == self.W:
            # flow time
            return (self.W >= self.Wq_bar + self.ts)
        
        if var == self.L:
            # inventory
            pass
        if var == self.Lq:
            # inventory in queue
            pass
        

In [18]:
# test setting variables equal to other variables
v1 = Variable('v1', 5)
v2 = Variable('v2', 6)
v = VectorVariable(2, 'v', [v1, v2])

this seems to work

In [21]:
mtest = Model(np.sum(v))

throws an error when actually trying to perform the substitution

### Line
_(may not be needed. wait to implement...)_

- contains the rate and routing information of the product as well as the cells
- ignores the constraints generated by the single-product context

__Line__
- based off of the single-class context
  - How much can we reuse?

we may not need this...

- [ ] test

In [3]:
class MCLine(Model):
    '''a multi-product production line
    
    Attributes
    ----------
    cells : ordered dict 
        the production cells
        
    processes
    '''
    
    def setup(self):
        '''
        Has to contain all of the resources
        
        Return only the relevant constraints
        
        Variable
        --------
        lambda   
        '''